#### Code for NWchem-Qiskit Driver
##### 1. import package

In [1]:
import os 
import sys
import yaml
import os
from qiskit_nature_qe import nwchem_driver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.exceptions import QiskitError
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD, UCC
import numpy as np
from nwchem2yaml import extract_fields

##### 2. nwchem output to yaml

In [2]:
data = extract_fields('qe_files/n2/output/demo.out')

with open('qe_files/n2/output/demo.yaml', 'w') as f:
    f.write(yaml.dump(data, default_flow_style=False)) 

##### 3. data to Qiskit electron structure problem class

In [4]:
driver = nwchem_driver.NWchem_Driver('qe_files/n2/output/demo.yaml')
es_problem = driver.run()
# hamiltonian  = es_problem.hamiltonian
# print(hamiltonian.second_q_op())

h_ij up-down equal: True
eri up-down equal: True
eri up-(down-up) equal: False
eri (up-down)-(down-up) equal: True


##### 4. Mapping Hamiltonian to qubit

In [5]:
mapper = JordanWignerMapper()

##### 5. specify ansatz

In [6]:
ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
        ),
    )

##### 6. Constructing VQE and check results 

In [7]:
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
# vqe_solver.initial_point = [0.0] * ansatz.num_parameters
vqe_solver.initial_point = np.random.rand(ansatz.num_parameters)
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.113647954101
  - computed part:      -1.113647954101
~ Nuclear repulsion energy (Hartree): 0.2074294884
> Total ground state energy (Hartree): -0.906218465701
